# Example step by step DLC

First create the project

In [ ]:
import os

os.system("module load cuDNN")
os.system("module list")

In [ ]:
# os.environ["LD_LIBRARY_PATH"] = "/nemo/apps/eb/software/cuDNN/8.4.1.50-CUDA-11.7.0/lib:/nemo/apps/eb/software/CUDA/11.7.0/nvvm/lib64:/nemo/apps/eb/software/CUDA/11.7.0/extras/CUPTI/lib64:/nemo/apps/eb/software/CUDA/11.7.0/lib"
# os.environ["LD_LIBRARY_PATH"] += ":/nemo/home/blota/.conda/envs/dlc/lib/"
# print(os.environ["LD_LIBRARY_PATH"])

In [ ]:
import tensorflow

print(tensorflow.__version__)
from tensorflow.python.client import device_lib


def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]


print(get_available_devices())

In [ ]:
import deeplabcut
from pathlib import Path

model = "headfixed_track_eye"
project = "blota_onix_pilote"

# project = "hey2_3d-vision_foodres_20220101"
# model = "headfixed_detect_eye"

config = Path("/nemo/lab/znamenskiyp/home/shared/projects/DLC_models/")
config /= f"{model}/config.yaml"

print(config)


In [ ]:
import flexiznam as flm

flm_sess = flm.get_flexilims_session(project_id=project)

In [ ]:
# get a list of videos to add
session_dict = {
    # "BRAC7448.2d": ["S20230421", "S20230412"],
    "BRYA142.5d": ["S20231002"],
    # "PZAH10.2f": ["S20230817"],
    # "PZAG3.4f": ["S20220421", "S20220517"],
}
from flexiznam.schema import Dataset
import shutil

DEINTERLEAVED = False
SPHERES = False
tmp_folder = Path("/nemo/lab/znamenskiyp/scratch/video_for_dlc_training")
tmp_folder.mkdir(exist_ok=True)
vids = []

for mouse, sessions in session_dict.items():
    for sess in sessions:
        fl_sess = flm.get_entity(
            name=f"{mouse}_{sess}", flexilims_session=flm_sess, datatype="session"
        )
        if fl_sess is None:
            print(f"Session {mouse}_{sess} not found")
        ds_dict = flm.get_datasets_recursively(
            origin_id=fl_sess.id, flexilims_session=flm_sess, dataset_type="camera"
        )
        ds = []
        for k, v in ds_dict.items():
            ds.extend(v)
        eye_ds = [d for d in ds if "eye_camera" in d.dataset_name]
        if DEINTERLEAVED:
            eye_ds = [d for d in ds if "deinterleaved" in d.dataset_name]
        else:
            eye_ds = [
                d
                for d in ds
                if ("left" in d.dataset_name) or ("right" in d.dataset_name)
            ]

        for ds in eye_ds:
            if SPHERES:
                if "Spheres" not in ds.genealogy[-2]:
                    print(ds.genealogy[-2])
                    continue
            vid = ds.path_full / ds.extra_attributes["video_file"]
            target = tmp_folder / f"{'_'.join(ds.genealogy[:-1])}_{vid.name}"
            if not target.exists():
                print(f"Copying {vid} to {target}")
                shutil.copy(vid, target)
            vids.append(target)
vids


In [ ]:
# vids = vids[:2]
vids

In [ ]:
from pathlib import Path
import shutil

tmp_folder = Path("/nemo/lab/znamenskiyp/home/shared/projects/DLC_models/tmp")
vids_name = [
    "PZAH6.4b_S20220526_R184428_SpheresPermTubeReward_left_eye_camera_dlc_tracking_uncropped_0_diagnostic_cropping.png",
    "PZAH6.4b_S20220523_R191440_SpheresPermTubeRewardPlayback_left_eye_camera_dlc_tracking_uncropped_0_diagnostic_cropping.png",
    "PZAG3.4f_S20220422_R130302_SpheresPermTubeReward_left_eye_camera_dlc_tracking_uncropped_0_diagnostic_cropping.png",
]
suffix = "_dlc_tracking_uncropped_0_diagnostic_cropping.png"
vids = []
for v in vids_name:
    flm_name = v[: -len(suffix)]
    ds = flm.Dataset.from_flexilims(name=flm_name, flexilims_session=flm_sess)
    vid = ds.path_full / ds.extra_attributes["video_file"]
    target = tmp_folder / f"{'_'.join(ds.genealogy[:-1])}_{vid.name}"
    if not target.exists():
        print(f"Copying {vid} to {target}")
        shutil.copy(vid, target)
    vids.append(target)
vids

In [ ]:
# add these videos (do it only once)
if True:
    deeplabcut.add_new_videos(
        config,
        videos=[str(v) for v in vids],
        extract_frames=False,
        copy_videos=True,
    )

In [ ]:
copied_vids = [config.parent / "videos" / v.name for v in vids]
copied_vids

In [ ]:
# this will reextract for all videos
if True:
    deeplabcut.extract_frames(
        config=config,
        userfeedback=True,
        # videos_list=[str(v) for v in copied_vids],
        algo="uniform",
        crop=True,
    )


Edit config file manually to add the skeleton and body parts you want. Then head to the
VM to label manually. Copy the labelled data back to camp. ~~edit config.yaml to change
paths if needed~~.

Then check labels

In [ ]:
deeplabcut.check_labels(config, visualizeindividuals=True)

In [ ]:
_ = deeplabcut.create_training_dataset(
    config, augmenter_type="imgaug", userfeedback=False
)

Now train the network (unsing the sbatch script)

In [ ]:
o = deeplabcut.evaluate_network(config, Shuffles=[1], plotting=True)

In [ ]:
scm = deeplabcut.extract_save_all_maps(config, shuffle=1, Indices=[0, 5])

# Label video

Now you can label the video


In [ ]:
config


In [ ]:
video_path = [
    "/camp/lab/znamenskiyp/home/shared/projects/blota_onix_pilote/BRYA142.5d/S20231002/R121600_HF_camera/righteye_camera_dlc_tracking_cropped_0/righteye_camera.mp4"
]
deeplabcut.create_labeled_video(config, video_path, videotype=".mp4")